In [1]:
import requests
import json
from bs4 import BeautifulSoup
import itertools
import pprint as pp
import time
from datetime import datetime
import pandas as pd

In [2]:
def get_car_links(url_ads_page):
    """
    Функция, получает страницу с объявлениями, возвращает - список url объявлений на этой странице
    """
    ads_list = []
    page = requests.get(url_ads_page)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if_last_page = 0

    if len(soup.find_all('a', class_='Button_disabled')) == 0:
        if_last_page = 1
    
    for el in soup.find_all('a', class_='Button_disabled'):
        if ('Следующая' in el.text) or (page.status_code == 404):
            if_last_page = 1
    
    for link in soup.find_all('a', class_='Link ListingItemTitle-module__link'):
        ads_list.append(link['href'])
        
    return ads_list, if_last_page

In [24]:
def parse_ad(url):
    """
    Функция - парсит необходимые данные из объявления
    Возвращает словарь с данными
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    script = soup.find_all('script',id = "initial-state")[0].contents[0]
    data = json.loads(script)
    
    # словарь с данными объявления
    car_dict = {'bodyType':0, 'brand':0, 'car_url': 0, 'color':0, 'complectation_dict':0, 
                'description':0, 'engineDisplacement':0, 'enginePower':0, 'equipment_dict':0,
                'fuelType':0, 'image':0, 'mileage':0,  'modelDate':0, 'model_info':0, 
                'model_name':0, 'name':0, 'numberOfDoors':0, 'parsing_unixtime':0, 'priceCurrency':0,
                'productionDate':0, 'sell_id':0, 'super_gen':0, 
                'vehicleConfiguration':0, 'vehicleTransmission':0,'vendor':0, 
                'Владельцы':0,'Владение':0, 'ПТС':0,    
                 'Привод':0, 'Руль':0,  'Состояние':0,   'Таможня':0, 'price':0}
    
    # начинаем парсить
    try: car_dict['bodyType'] = data['card']['vehicle_info']['configuration']['human_name'] 
    except: car_dict['bodyType'] = None

    try: car_dict['brand'] = data['card']['vehicle_info']['mark_info']['code']
    except: car_dict['brand'] = None

    car_dict['car_url'] = url

    try: car_dict['color'] = soup.find_all('li', class_='CardInfoRow CardInfoRow_color')[0].find('a').text
    except: car_dict['color'] = None

    try: car_dict['complectation_dict'] = data['card']['vehicle_info']['configuration']
    except: car_dict['complectation_dict'] = None

    try: car_dict['description'] = data['card']['description']
    except: car_dict['description'] = None

    try: car_dict['engineDisplacement'] = data['card']['vehicle_info']['tech_param']['human_name'].split()[0]
    except: car_dict['engineDisplacement'] = None

    try: car_dict['enginePower'] = data['card']['vehicle_info']['tech_param']['power']
    except: car_dict['enginePower'] = None

    try: car_dict['equipment_dict'] = data['card']['vehicle_info']['equipment']
    except: car_dict['equipment_dict'] = None

    try: car_dict['fuelType'] = data['card']['lk_summary'].split()[-1]
    except: car_dict['fuelType'] = None

    try: car_dict['image'] = 'http:' + data['card']['vehicle_info']['configuration']['main_photo']['sizes']['orig']
    except: car_dict['image'] = None

    try: car_dict['mileage'] = data['card']['state']['mileage']
    except: car_dict['mileage'] = None

    try: car_dict['modelDate'] = data['card']['vehicle_info']['super_gen']['year_from']
    except: car_dict['modelDate'] = None

    try: car_dict['model_info'] = data['card']['vehicle_info']['model_info']
    except: car_dict['model_info'] = None

    try: car_dict['model_name'] = data['card']['vehicle_info']['model_info']['code']
    except: car_dict['model_name'] = None

    try: car_dict['name'] =  data['card']['vehicle_info']['tech_param']['human_name']
    except: car_dict['name'] = None

    try: car_dict['numberOfDoors'] = data['card']['vehicle_info']['configuration']['doors_count']
    except: car_dict['numberOfDoors'] = None

    car_dict['parsing_unixtime'] = time.time()

    try: car_dict['priceCurrency'] = data['card']['price_info']['currency']
    except: car_dict['priceCurrency'] = None

    try: car_dict['productionDate'] = data['card']['documents']['year']
    except: car_dict['productionDate'] = None

    try: car_dict['sell_id'] = data['card']['id']
    except: car_dict['sell_id'] = None

    try: car_dict['super_gen'] = data['card']['vehicle_info']['super_gen']
    except: car_dict['super_gen'] = None

    try: car_dict['vehicleConfiguration'] = data['card']['vehicle_info']['configuration']['body_type'] + ' '\
        + data['card']['vehicle_info']['tech_param']['transmission'] + ' '\
        + data['card']['vehicle_info']['tech_param']['human_name'].split()[0]
    except: car_dict['vehicleConfiguration'] = None

    try: car_dict['vehicleTransmission'] = soup.\
        find_all('li', class_='CardInfoRow CardInfoRow_transmission')[0].find_all('span')[1].text
    except: car_dict['vehicleTransmission'] = None

    try: car_dict['vendor'] = data['card']['vehicle_info']['vendor']
    except: car_dict['vendor'] = None

    try: car_dict['Владельцы'] = soup.find_all(
        'li', class_='CardInfoRow CardInfoRow_ownersCount')[0].find_all('span')[1].text
    except: car_dict['Владельцы'] = None

    try: car_dict['Владение'] =  soup.\
        find_all('li', class_='CardInfoRow_owningTime')[0].find_all('span')[1].text
    except: car_dict['Владение'] = None

    try: car_dict['ПТС'] = soup.\
        find_all('li', class_='CardInfoRow_pts')[0].find_all('span')[1].text
    except: car_dict['ПТС'] = None

    try: car_dict['Привод'] = data['card']['lk_summary'].split(', ')[-2]
    except: car_dict['Привод'] = None

    try: car_dict['Руль'] = soup.find_all(
        'li', class_='CardInfoRow CardInfoRow_wheel')[0].find_all('span')[1].text
    except: car_dict['Руль'] = None

    try: car_dict['Состояние'] = soup.find_all(
        'li', class_='CardInfoRow CardInfoRow_state')[0].find_all('span')[1].text
    except: car_dict['Руль'] = None

    try: car_dict['Таможня'] = soup.find_all(
        'li', class_='CardInfoRow CardInfoRow_customs')[0].find_all('span')[1].text
    except: car_dict['Таможня'] = None

    try: car_dict['price'] = data['card']['price_info']['price']
    except: car_dict['price'] = None

    try: car_dict['start_date'] =  data['card']['additional_info']['hot_info']['start_time']
    except: car_dict['start_date'] = None

    try: car_dict['hidden'] =  data['card']['additional_info']['hidden']
    except: car_dict['hidden'] = None
    
    # возвращаем словарь
    return car_dict

In [21]:
pd.set_option('display.max_columns', None)
test = pd.read_csv('test.csv')
test[test['model_name']=='PASSAT'].sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
23761,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,синий,NaN,Безопасность: 6. Обзор: 3. Мультимедиа: 3. Ком...,1.8 LTR,152 N12,"{""asr"":true,""ptf"":true,""airbag-driver"":true,""a...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,116332,2011,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.8 AMT (152 л.с.),4,1603215006,RUB,2011,1101191135,"{""id"":""7150232"",""displacement"":1798,""engine_ty...",SEDAN ROBOT 1.8,роботизированная,EUROPEAN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
26293,универсал 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,серый,NaN,Доброго времени суток! Продаю VW Passat b4 уни...,2.0 LTR,115 N12,"{""electro-window-back"":true,""engine-proof"":tru...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,340300,1993,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,2.0 MT (115 л.с.),5,1603224623,RUB,1996,1101220227,"{""id"":""6381424"",""displacement"":1984,""engine_ty...",WAGON_5_DOORS MECHANICAL 2.0,механическая,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
23679,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,серебристый,NaN,",\nБезопасная покупка в АСЦ/ЧЕСТНО –легко! \n\...",1.8 LTR,152 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,150413,2005,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.8 AMT (152 л.с.),4,1603214695,RUB,2010,1101231042,"{""id"":""4553930"",""displacement"":1798,""engine_ty...",SEDAN ROBOT 1.8,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
30679,универсал 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,чёрный,NaN,авто в отличном состоянии 1год из германии про...,1.6 LTR,120 N12,"{""engine-proof"":true,""asr"":true,""tinted-glass""...",дизель,https://autoru.naydex.net/cSRr96A18/708ec0RhY5...,130000,2014,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.6d MT (120 л.с.),5,1603575083,RUB,2015,1101318377,"{""id"":""20273230"",""displacement"":1598,""engine_t...",WAGON_5_DOORS MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,11 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
23737,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,чёрный,NaN,Один собственник\nОригинальный ПТС\nПодтвержде...,1.4 LTR,150 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,36051,2014,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.4 AMT (150 л.с.),4,1603214912,RUB,2018,1101201871,"{""id"":""20581538"",""displacement"":1395,""engine_t...",SEDAN ROBOT 1.4,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [25]:
# протестируем функции и сравним с тестовыми данными
ads_list = []

for url in get_car_links('https://auto.ru/moskva/cars/volkswagen/passat/all/?page=2')[0]:
    ads_list.append(parse_ad(url))
    #time.sleep(1) # На всякий случай ждем 1 секунду

In [26]:
df = pd.DataFrame(ads_list)
df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,start_date,hidden
0,Седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,чёрный,"{'id': '2309872', 'body_type': 'SEDAN', 'doors...",Volkswagen Passat B6 Седан 1.6 MT (102 л.с.).О...,1.6,102,"{'engine-proof': True, 'cruise-control': True,...",бензин,http://avatars.mds.yandex.net/get-verba/103038...,154345,2005,"{'code': 'PASSAT', 'name': 'Passat', 'ru_name'...",PASSAT,1.6 MT (102 л.с.),4,1.611668e+09,RUR,2008,1102217414,"{'id': '2309871', 'name': 'B6', 'year_from': 2...",SEDAN MECHANICAL 1.6,механическая,EUROPEAN,3 или более,11 лет и 12 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,360000.0,2021-01-19T17:57:07Z,False
1,Седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,белый,"{'id': '20232338', 'body_type': 'SEDAN', 'door...",- Автомобиль с полным НДС!\n- Один владелец\n-...,1.4,150,"{'start-button': True, 'multi-wheel': True, 'p...",бензин,http://avatars.mds.yandex.net/get-verba/787013...,41445,2014,"{'code': 'PASSAT', 'name': 'Passat', 'ru_name'...",PASSAT,1.4 AMT (150 л.с.),4,1.611668e+09,RUR,2016,1102266881,"{'id': '20232305', 'name': 'B8', 'year_from': ...",SEDAN ROBOT 1.4,роботизированная,EUROPEAN,1 владелец,None,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1458000.0,2021-01-25T09:58:37Z,False
2,Седан,VOLKSWAGEN,https://auto.ru/cars/new/group/volkswagen/pass...,None,"{'id': '21674468', 'body_type': 'SEDAN', 'door...",Эксклюзивные условия в Фольксваген Центр Север...,1.4,150,"{'asr': True, 'esp': True, 'usb': True, 'multi...",бензин,http://avatars.mds.yandex.net/get-verba/787013...,0,2019,"{'code': 'PASSAT', 'name': 'Passat', 'ru_name'...",PASSAT,1.4 AMT (150 л.с.),4,1.611668e+09,RUR,2020,1101835509,"{'id': '21674342', 'name': 'B8 Рестайлинг', 'r...",SEDAN ROBOT 1.4,None,EUROPEAN,None,None,None,передний,None,0,None,2393000.0,2020-12-08T12:34:36Z,False
3,Седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,синий,"{'id': '7150227', 'body_type': 'SEDAN', 'doors...",Автомобиль в отличном состоянии. Бережная эксп...,1.8,152,"{'engine-proof': True, 'tinted-glass': True, '...",бензин,http://avatars.mds.yandex.net/get-verba/787013...,96000,2011,"{'code': 'PASSAT', 'name': 'Passat', 'ru_name'...",PASSAT,1.8 AMT (152 л.с.),4,1.611668e+09,RUR,2011,1102272685,"{'id': '7150225', 'name': 'B7', 'year_from': 2...",SEDAN ROBOT 1.8,роботизированная,EUROPEAN,3 или более,None,Оригинал,передний,Левый,Не требует ремонта,Растаможен,659000.0,2021-01-25T18:07:48Z,False
4,Седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,синий,"{'id': '7150227', 'body_type': 'SEDAN', 'doors...",Машина в отличном состоянии!\nПолностью обслуж...,1.8,152,{},бензин,http://avatars.mds.yandex.net/get-verba/787013...,89000,2011,"{'code': 'PASSAT', 'name': 'Passat', 'ru_name'...",PASSAT,1.8 AMT (152 л.с.),4,1.611668e+09,RUR,2012,1101973042,"{'id': '7150225', 'name': 'B7', 'year_from': 2...",SEDAN ROBOT 1.8,роботизированная,EUROPEAN,2 владельца,None,Оригинал,передний,Левый,Не требует ремонта,Растаможен,895000.0,2020-12-21T18:36:36Z,False


In [27]:
models_dict = {}
for brand in test['brand'].unique():
    model_list = []
    for model in test['model_name'][test['brand'] == brand].unique():
        model_list.append(model.lower())
    models_dict[brand.lower()] = model_list

In [30]:
models_dict.keys()

dict_keys(['skoda', 'audi', 'honda', 'volvo', 'bmw', 'nissan', 'infiniti', 'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi'])

In [10]:
ads_list = []
i = 0

for brand in  models_dict.keys():
    if len(ads_list) > 0:
        print('В марку {} было добавлено {} объявлений\n\n'.format(prev_brand_name, len(ads_list)))
        backup_df = pd.DataFrame(ads_list)
        backup_df.to_csv('soup_parser/' + prev_brand_name + '_backup.csv')
        ads_list = [] # на всякий случай будем чистить для каждого бренда
    print('Start {} {}'.format(brand, datetime.now()))
    for model in models_dict[brand]:
        print('\tModel {} {}'.format(model, datetime.now()))
        ads_num = 0
        for i in itertools.count(start = 1):
            ads_page = 'https://auto.ru/moskva/cars/' + brand + '/' + model + '/all/?page=' + str(i)
            if requests.get(ads_page).status_code == 404:
                break
            #print(ads_page)
            result = get_car_links(ads_page)
            try:
                for url in result[0]:
                    ads_list.append(parse_ad(url))
            except:
                pass
                    #time.sleep(1) # На всякий случай ждем 1 секунду
            ads_num += len(result[0])
            if result[1] == 1:
                backup_df = pd.DataFrame(ads_list)
                backup_df.to_csv('soup_parser/' + brand + '_' + model + '_backup.csv')
                print('Было добавлено {} страниц и {} объявлений'.format(i, ads_num) )
                break
    df = pd.DataFrame(ads_list)
    result_df = result_df.append(df)
    result_df.to_csv(brand + '_backup.csv')
    prev_brand_name = brand

Start mitsubishi 2021-01-22 01:29:20.493739
	Model pajero_sport 2021-01-22 01:29:20.494737
Было добавлено 6 страниц и 223 объявлений
	Model outlander 2021-01-22 01:31:07.137819
Было добавлено 27 страниц и 971 объявлений
	Model l200 2021-01-22 01:38:41.941220
Было добавлено 2 страниц и 50 объявлений
	Model montero 2021-01-22 01:39:04.492337
Было добавлено 1 страниц и 21 объявлений
	Model lancer 2021-01-22 01:39:15.346655
Было добавлено 14 страниц и 497 объявлений
	Model pajero 2021-01-22 01:42:48.998123
Было добавлено 7 страниц и 225 объявлений
	Model montero_sport 2021-01-22 01:44:29.543922
Было добавлено 1 страниц и 27 объявлений
	Model galant 2021-01-22 01:44:42.186137
Было добавлено 3 страниц и 110 объявлений
	Model pajero_mini 2021-01-22 01:45:30.237552
Было добавлено 1 страниц и 3 объявлений
	Model pajero_io 2021-01-22 01:45:32.425820
Было добавлено 1 страниц и 9 объявлений
	Model asx 2021-01-22 01:45:37.733223
Было добавлено 16 страниц и 566 объявлений
	Model colt 2021-01-22 01:4

Было добавлено 1 страниц и 9 объявлений
	Model caravan_coach 2021-01-22 02:28:26.350253
Было добавлено 1 страниц и 0 объявлений
	Model datsun 2021-01-22 02:28:27.601904
Было добавлено 1 страниц и 4 объявлений
	Model gloria 2021-01-22 02:28:30.514682
Было добавлено 1 страниц и 2 объявлений
	Model expert 2021-01-22 02:28:32.511789
Было добавлено 1 страниц и 1 объявлений
	Model 100nx 2021-01-22 02:28:33.982360
Было добавлено 1 страниц и 2 объявлений
	Model safari 2021-01-22 02:28:36.316271
Было добавлено 1 страниц и 4 объявлений
	Model liberty 2021-01-22 02:28:39.340757
Было добавлено 1 страниц и 4 объявлений
	Model cedric 2021-01-22 02:28:42.514834
Было добавлено 1 страниц и 11 объявлений
	Model vanette 2021-01-22 02:28:48.346850
Было добавлено 1 страниц и 4 объявлений
	Model sunny 2021-01-22 02:28:51.171363
Было добавлено 1 страниц и 10 объявлений
	Model otti 2021-01-22 02:28:56.221968
Было добавлено 1 страниц и 1 объявлений
	Model dayz_roox 2021-01-22 02:28:57.817263
Было добавлено 1 с

Было добавлено 1 страниц и 0 объявлений
	Model noah 2021-01-22 03:13:36.823688
Было добавлено 1 страниц и 9 объявлений
	Model lite_ace 2021-01-22 03:13:41.347143
Было добавлено 1 страниц и 7 объявлений
	Model blade 2021-01-22 03:13:45.708988
Было добавлено 1 страниц и 2 объявлений
	Model picnic 2021-01-22 03:13:47.390241
Было добавлено 1 страниц и 2 объявлений
	Model corolla_spacio 2021-01-22 03:13:49.176444
Было добавлено 1 страниц и 6 объявлений
	Model carina_ed 2021-01-22 03:13:53.230469
Было добавлено 1 страниц и 1 объявлений
	Model corolla_levin 2021-01-22 03:13:54.553192
Было добавлено 1 страниц и 11 объявлений
	Model corolla_rumion 2021-01-22 03:13:59.697628
Было добавлено 1 страниц и 4 объявлений
	Model grand_hiace 2021-01-22 03:14:02.322352
Было добавлено 1 страниц и 1 объявлений
	Model carina_e 2021-01-22 03:14:03.686314
Было добавлено 1 страниц и 21 объявлений
	Model origin 2021-01-22 03:14:12.231008
Было добавлено 1 страниц и 0 объявлений
	Model aygo 2021-01-22 03:14:13.427

Было добавлено 1 страниц и 12 объявлений
	Model xc40 2021-01-22 03:57:57.968844
Было добавлено 2 страниц и 53 объявлений
	Model xc60 2021-01-22 03:58:27.155243
Было добавлено 12 страниц и 435 объявлений
	Model c30 2021-01-22 04:01:48.524688
Было добавлено 1 страниц и 23 объявлений
	Model v50 2021-01-22 04:01:58.804624
Было добавлено 1 страниц и 3 объявлений
	Model xc90 2021-01-22 04:02:01.143473
Было добавлено 9 страниц и 330 объявлений
	Model v90_cross_country 2021-01-22 04:04:52.801319
Было добавлено 1 страниц и 17 объявлений
	Model s60 2021-01-22 04:05:01.753476
Было добавлено 4 страниц и 143 объявлений
	Model xc70 2021-01-22 04:06:06.209528
Было добавлено 3 страниц и 88 объявлений
	Model v40_cc 2021-01-22 04:06:48.789014
Было добавлено 1 страниц и 30 объявлений
	Model s40 2021-01-22 04:07:03.421296
Было добавлено 3 страниц и 80 объявлений
	Model s90 2021-01-22 04:07:38.883017
Было добавлено 1 страниц и 23 объявлений
	Model v40 2021-01-22 04:07:49.831391
Было добавлено 1 страниц и 8

KeyError: 'bb'